In [1]:
import pandas as pd
import json
import numpy
from py2neo import Graph, Node

In [2]:
fileName = 'corecomp1.csv'

In [3]:
companyData = pd.read_csv(fileName, delimiter=',')

In [4]:
companyData.head()
#print(set(companyData['class_of_company']))

,cin,company_name,date_of_incorporation,class_of_company,company_category,company_subcategory,registration_number,roc_code,registered_address,email_id,company_status,number_of_members,date_of_last_agm,authorised_capital,whether_listed_or_not,date_of_balance_sheet,paid_up_capital
0,L00000CH1983PLC031318,SAB INDUSTRIES LIMITED,16/02/1983,Public,Company limited by Shares,Non-govt company,31318,RoC-Chandigarh,S.C.O. 49-50 SECTOR - 26 MADHYA MARG CHANDIG...,ssl_ssg@glide.net.in,Active,0,30/09/2016,2.000000e+08,Listed,31/03/2016,1.518840e+08
1,L00000CH1990PLC010573,LAKSHMI ENERGY AND FOODS LIMITED,20/07/1990,Public,Company limited by Shares,Non-govt company,10573,RoC-Chandigarh,SCO 18 19 1ST FLOORSECTOR 9 D CHANDIGARH U T C...,corporate@lakshmigroup.in,Active,0,28/09/2016,2.000000e+08,Listed,31/03/2016,1.399400e+08
2,L00000CH1994PLC015150,KOHINOOR BROADCASTING CORPORATION LIMITED,11/10/1994,Public,Company limited by Shares,Non-govt company,15150,RoC-Chandigarh,SCO 87 2nd Floor Sector 46-C Chandigarh Chan...,mail@kohinoorbroadcasting.com,Active,0,23/09/2014,1.160000e+09,Listed,31/03/2014,1.102060e+09
3,L00000HP1994PLC014770,HITKARI INDUSTRIES LIMITED,29/06/1994,Public,Company limited by Shares,Non-govt company,14770,RoC-HimachalPradesh,PLOT NO 18 SECTOR 1INDUSTRIAL AREA PARWANOO SO...,sachdevakk@hotmail.com,Active,0,30/09/2015,6.000000e+07,Listed,31/03/2015,3.205470e+07
4,L00000MH1946PLC197474,QUADRANT TELEVENTURES LIMITED,02/08/1946,Public,Company limited by Shares,Non-govt company,197474,RoC-Mumbai,AUTOCARS COMPOUND ADALAT ROAD AURANGABAD MH 43...,secretarial@infotelconnect.com,Active,0,21/07/2016,1.500000e+10,Listed,31/03/2016,6.122600e+08


In [6]:
graph = Graph()

In [7]:
company = graph.data('match(c:company) return c limit 5')
for c in company:
    c = c['c']
    for key in c:
        print(key,type(c[key]))
    print()

name <class 'str'>
companynature <class 'str'>
subcategory <class 'str'>
incorp_date <class 'str'>
uuid <class 'int'>
aliases <class 'list'>
address <class 'str'>
authcap <class 'str'>
listing <class 'str'>
type <class 'str'>
totalincome <class 'float'>
status <class 'str'>
email <class 'str'>
paidupcap <class 'str'>
regauth <class 'str'>
profit <class 'float'>
cin <class 'str'>
regnum <class 'str'>

name <class 'str'>
companynature <class 'str'>
subcategory <class 'str'>
incorp_date <class 'str'>
busscategory <class 'str'>
uuid <class 'int'>
aliases <class 'list'>
address <class 'str'>
authcap <class 'str'>
listing <class 'str'>
type <class 'str'>
totalincome <class 'float'>
status <class 'str'>
email <class 'str'>
paidupcap <class 'str'>
regauth <class 'str'>
profit <class 'float'>
cin <class 'str'>
regnum <class 'str'>

name <class 'str'>
companynature <class 'str'>
subcategory <class 'str'>
incorp_date <class 'str'>
busscategory <class 'str'>
uuid <class 'int'>
aliases <class 'list

In [8]:
print(companyData.columns)

Index(['cin', 'company_name', 'date_of_incorporation', 'class_of_company',
       'company_category', 'company_subcategory', 'registration_number',
       'roc_code', 'registered_address', 'email_id', 'company_status',
       'number_of_members', 'date_of_last_agm', 'authorised_capital',
       'whether_listed_or_not', 'date_of_balance_sheet', 'paid_up_capital'],
      dtype='object')


In [9]:
rename = {}
rename['cin'] = 'cin'
rename['company_name'] = 'name'
rename['date_of_incorporation'] = 'incorp_date'
rename['email_id'] = 'email'
rename['whether_listed_or_not'] = 'listing'
rename['company_subcategory'] = 'subcategory'
rename['company_category'] = 'companynature'
rename['authorised_capital'] = 'authcap'
rename['registered_address'] = 'address'
rename['registration_number'] = 'regnum'
rename['class_of_company'] = 'type'
rename['company_status'] = 'status'
rename['paid_up_capital'] = 'paidupcap'
rename['roc_code'] = 'regauth'

In [10]:
companyData = companyData.rename(columns=rename)

In [11]:
attributes = list(rename.values())

In [12]:
print(attributes)

['incorp_date', 'regnum', 'regauth', 'authcap', 'type', 'email', 'address', 'status', 'companynature', 'paidupcap', 'listing', 'subcategory', 'cin', 'name']


In [13]:
maxUUIDQuery = 'match (n) return max(n.uuid) as uuid'
result = graph.data(maxUUIDQuery)
maxUUID = result[0]['uuid']
print(maxUUID)

208928


In [14]:
#verification
countCompanies = 'match (c:company) return count(distinct c) as c'
print('Number of companies in database : ', graph.data(countCompanies) )

Number of companies in database :  [{'c': 64226}]


In [15]:
#changing roc code from RoC-state to state
companyData['regauth'] = companyData['regauth'].apply(lambda x: x.split('-')[1])

In [16]:
companyData.shape

(58817, 17)

In [17]:
uuid = maxUUID + 1
shape = companyData.shape
print(uuid, shape)
checkIfExist = 'match (c:company) where c.cin="{}" return c'
companyPushQuery = 'merge (c1:company {{cin : "{}"}}) set c1={{param}} return c1'
removeQuery = 'match (c:company) where c.uuid > {} delete c'
for i in range(shape[0]):    
    params = {'param':{}}
    for attr in attributes:
        params['param'][attr] = str(companyData.loc[i][attr])
    params['param']['uuid'] = uuid
    #for k in params['param']:
    #    print(k, type(params['param'][k]))
    result = graph.data(checkIfExist.format(companyData.loc[i]['cin']))
    #print(result)
    if len(result) == 0:
        #print(i,' Does not Exist')
        uuid += 1
    else:
        continue
    #print(result)
    graph.run(companyPushQuery.format(companyData.loc[i]['cin']), params)
    if i%1000 == 0:
        print(i, 'done')
    #print(graph.data(removeQuery.format(maxUUID)))

208929 (58817, 17)
12000 done
13000 done
16000 done
17000 done
21000 done
22000 done
24000 done
27000 done
28000 done
29000 done
30000 done
32000 done
35000 done
40000 done
41000 done
42000 done
43000 done
45000 done
49000 done
50000 done
51000 done
53000 done
56000 done
58000 done


In [18]:
print(uuid)

238116


In [19]:
#verification
countCompanies = 'match (c:company) return count(distinct c)'
print('Number of companies in database : ' , graph.data(countCompanies))

Number of companies in database :  [{'count(distinct c)': 93413}]


In [20]:
uuid - maxUUID

29188